## Go

Instalar Go

In [ ]:
!apt install golang-go

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  golang-1.18-go golang-1.18-src golang-src
Suggested packages:
  bzr | brz mercurial subversion
The following NEW packages will be installed:
  golang-1.18-go golang-1.18-src golang-go golang-src
0 upgraded, 4 newly installed, 0 to remove and 45 not upgraded.
Need to get 82.3 MB of archives.
After this operation, 436 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 golang-1.18-src all 1.18.1-1ubuntu1.1 [16.2 MB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 golang-1.18-go amd64 1.18.1-1ubuntu1.1 [66.0 MB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 golang-src all 2:1.18~0ubuntu2 [4,438 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 golang-go amd64 2:1.18~0ubuntu2 [41.8 kB]
Fetched 82.3 MB in 4s (21.6 MB/s)
Selecting previously unselected pa

In [ ]:
%%writefile hilos.go

package main

import (
	"fmt"
	"math/rand"
	"os"
	"reflect"
	"strconv"
	"sync"
	"time"
)

var (
	initialMatrix    [][]int
	sequentialMatrix [][]int
	concurrentMatrix [][]int
)

const (
	showMatrices    = false
	maxThreads      = 300
	maxSize         = 10000
	maxVisibleSize  = 50
	maxRandomNumber = 10
	dimensionPos    = 1
	threadsPos      = 2
)

func main() {
	//testing the program with arguments.
	if len(os.Args) == 3 {
		sizeToTest, err := strconv.Atoi(os.Args[dimensionPos])
		if err != nil {
			panic(err.Error())
		}

		threadsToTest, err := strconv.Atoi(os.Args[threadsPos])
		if err != nil {
			panic(err.Error())
		}

		scalarToTest := 10

		prevalidateArguments(&sizeToTest, &threadsToTest)
		initMatrices(sizeToTest)

		sequentialStart := time.Now()
		multiplyMatrixSequential(scalarToTest)
		sequentialEnd := time.Since(sequentialStart)

		concurrentStart := time.Now()
		multiplyMatrixConcurrent(scalarToTest, threadsToTest)
		concurrentEnd := time.Since(concurrentStart)

		equal := compareMatrices(sequentialMatrix, concurrentMatrix)
		fmt.Println("The matrices are equal: ", equal)

		if showMatrices && sizeToTest < maxVisibleSize {
			printMatrix(initialMatrix)
			printMatrix(sequentialMatrix)
			printMatrix(concurrentMatrix)
		}

		fmt.Println("Time elapsed for sequential: ", sequentialEnd.Milliseconds(), "ms")
		fmt.Println("Time elapsed for concurrent: ", concurrentEnd.Milliseconds(), "ms")

	} else { //testing the program with the matrix
		threads := []int{0, 1, 2, 4}
		sizes := []int{10, 20, 40, 80}
		scalar := 10
		results := measureTimes(threads, sizes, scalar)
		printResults(results, threads, sizes)
	}
}

func printResults(results [][]time.Duration, threads []int, sizes []int) {
	for idxSize, row := range results {
		fmt.Print("N:", sizes[idxSize], "\n\t")
		for idxThread := range row {
			if idxThread == 0 {
				fmt.Print("Secuencial:", results[idxSize][idxThread], "\t")
			} else {
				fmt.Print(threads[idxThread], " Hilos:", results[idxSize][idxThread], "\t")
			}
		}
		fmt.Println()
	}
}

func measureTimes(threads []int, sizes []int, scalar int) (results [][]time.Duration) {
	results = make([][]time.Duration, len(sizes), len(threads))
	for idxSize, size := range sizes {
		initMatrices(size)
		for _, thread := range threads {
			times := measureTime(size, thread, scalar)
			results[idxSize] = append(results[idxSize], times)
		}
	}
	return
}

func measureTime(size int, threads int, scalar int) time.Duration {
	prevalidateArguments(&size, &threads)
	start := time.Now()
	if threads == 0 {
		multiplyMatrixSequential(scalar)
	} else {
		multiplyMatrixConcurrent(scalar, threads)
	}

	return time.Since(start)
}

func prevalidateArguments(size *int, threads *int) {
	if *size > maxSize {
		*size = maxSize
	}
	//if there are more threads than rows, set the amount of threads to the amount of rows.
	if *threads > *size {
		*threads = *size
	}
	//if the amount of threads is still larger than the max amount, set the amount of threads to the max amount.
	if *threads > maxThreads {
		*threads = maxThreads
	}
}

func initMatrices(size int) {
	initialMatrix = initMatrixRandom(size)
	sequentialMatrix = initMatrixZeroes(size)
	concurrentMatrix = initMatrixZeroes(size)
}

func initMatrixRandom(size int) [][]int {
	mat := make([][]int, size)
	for idxRow := range mat {
		for i := 0; i < size; i++ {
			mat[idxRow] = append(mat[idxRow], rand.Intn(maxRandomNumber))
		}
	}
	return mat
}

func initMatrixZeroes(size int) [][]int {
	mat := make([][]int, size)
	for idxRow := range mat {
		for i := 0; i < size; i++ {
			mat[idxRow] = append(mat[idxRow], 0)
		}
	}
	return mat
}

func multiplyMatrixSequential(scalar int) {
	for idxRow, row := range initialMatrix {
		for idxCol, val := range row {
			sequentialMatrix[idxRow][idxCol] = scalar * val
		}
	}
}

func multiplyMatrixConcurrent(scalar int, threads int) {
	rowsPerThread := calculateRowsPerThread(len(initialMatrix), threads)
	var wg sync.WaitGroup
	initialRow := 0
	for t := 0; t < threads; t++ {
		wg.Add(1)
		go multiplyRowsConcurrent(scalar, initialRow, initialRow+rowsPerThread[t], &wg)
		initialRow += rowsPerThread[t]
	}

	defer wg.Wait()
}

func calculateRowsPerThread(rows int, threads int) []int {
	rpt := make([]int, threads)
	for i := 0; i < threads; i++ {
		size := int(rows / threads)
		if i < rows%threads {
			size++
		}
		rpt[i] = size
	}
	return rpt
}

func multiplyRowsConcurrent(scalar int, initialRow int, finalRow int, wg *sync.WaitGroup) {
	defer wg.Done()
	n := len(initialMatrix)
	for i := initialRow; i < finalRow; i++ {
		for j := 0; j < n; j++ {
			concurrentMatrix[i][j] = initialMatrix[i][j] * scalar
		}
	}
}

func compareMatrices(mat1 [][]int, mat2 [][]int) bool {
	return reflect.DeepEqual(mat1, mat2)

}

func printMatrix(mat [][]int) {
	for _, row := range mat {
		fmt.Println(row)
	}
	fmt.Println("")
}


Overwriting hilos.go


In [ ]:
!go run hilos.go

N:10
	Secuencial:545ns	1 Hilos:10.665µs	2 Hilos:3.551µs	4 Hilos:8.026µs	
N:20
	Secuencial:1.139µs	1 Hilos:6.764µs	2 Hilos:3.276µs	4 Hilos:4.582µs	
N:40
	Secuencial:3.779µs	1 Hilos:11.351µs	2 Hilos:8.068µs	4 Hilos:7.468µs	
N:80
	Secuencial:20.867µs	1 Hilos:29.961µs	2 Hilos:23.798µs	4 Hilos:24.351µs	


En un entorno local, la tabla se ve de esta manera:

```
N:1000
        Secuencial:2.0086ms     100 Hilos:991µs 200 Hilos:1.0029ms      400 Hilos:1.0043ms
N:2000
        Secuencial:15.0057ms    100 Hilos:3.419ms       200 Hilos:2.5861ms      400 Hilos:4.0017ms
N:4000
        Secuencial:57.0139ms    100 Hilos:13.3978ms     200 Hilos:13.5995ms     400 Hilos:13.4383ms
N:8000
        Secuencial:247.0807ms   100 Hilos:52.0007ms     200 Hilos:55.0151ms     400 Hilos:54.0174ms
```

Por lo cual puede verse una reducción en el tiempo de procesamiento al incrementar la cantidad de hilos.

In [ ]:
!go run hilos.go 4000 10

# command-line-arguments
./hilos.go:186:12: undefined: sizei


In [ ]:
!go run hilos.go 4000 50

# command-line-arguments
./hilos.go:186:12: undefined: sizei


In [ ]:
!go run hilos.go 4000 200

The matrices are equal:  true
Time elapsed for sequential:  32 ms
Time elapsed for concurrent:  34 ms


In [ ]:
!go run hilos.go 4000 1000

The matrices are equal:  true
Time elapsed for sequential:  31 ms
Time elapsed for concurrent:  33 ms



# JAVA

In [ ]:
%%writefile hilos.java

package main;

import java.util.ArrayList;
import java.util.Random;


/**
 * Ejecutar con parametros : n
 */
public class Hilos {

	private static int[][] inicial;
	private static int[][] secuencial;
	private static int[][] concurrente;
	private static int n = 0;
	private static final int ESCALAR = 5;
	private static ArrayList<Thread> threads = new ArrayList<Thread>();


	public static class MatrixThread implements Runnable {

		private int filaDesde;
		private int filaHasta;

		public MatrixThread(int filaDesde,int filaHasta) {
			this.filaDesde = filaDesde;
			this.filaHasta = filaHasta;
		}

		@Override
		public void run() {
			for(int i = filaDesde; i<=filaHasta;i++) {
				for(int j=0;j<n;j++) {
					concurrente[i][j] = inicial[i][j] * ESCALAR;
				}
			}
		}
	}

	public static void main(String[] args) {

		n = Integer.valueOf(args[0]);

		inicializarMatriz(n);
		long secTime = secuentialExecute();
		long concTimeOneThread = concurrentExecute(1);
		checkValidityOfMatrix();
		long concTimeTwoThreads = concurrentExecute(2);
		checkValidityOfMatrix();
		long concTimeFourThreads = concurrentExecute(4);
		checkValidityOfMatrix();

		System.out.println("Tiempo Secuencial : ".concat(String.valueOf(secTime)).concat(" milisegundos"));
		System.out.println("Tiempo 1 Thread   : ".concat(String.valueOf(concTimeOneThread)).concat(" milisegundos"));
		System.out.println("Tiempo 2 Threads  : ".concat(String.valueOf(concTimeTwoThreads)).concat(" milisegundos"));
		System.out.println("Tiempo 4 Threads  : ".concat(String.valueOf(concTimeFourThreads)).concat(" milisegundos"));

	}


	private static void checkValidityOfMatrix() {
		try {
			for (int i = 0; i < n; i++) {
				for (int j = 0; j < n; j++) {
					if(secuencial[i][j] != concurrente[i][j]) {
						throw new Exception("resultado erroneo");
					}
				}
			}
			System.out.println("Las matrices secuencial y concurrente son iguales");
		}catch(Exception ex) {
			System.out.println("Las matrices secuencial y concurrente son distintas");
		}
	}


	private static long concurrentExecute(int cantHilos) {
		int filas_por_hilos = n / cantHilos;
		for(int i = 1;i <= cantHilos;i++) {
			int fila_desde = (i - 1) * filas_por_hilos;
			int fila_hasta = (i * filas_por_hilos) -1;
			threads.add(new Thread(new MatrixThread(fila_desde,fila_hasta)));
		}

		long initConcTime = System.currentTimeMillis();
		threads.stream().forEach(t -> t.start());


		threads.stream().forEach(t -> {
			try {
				t.join();
			} catch (InterruptedException e) {
				e.printStackTrace();
			}
		});
		long endConcTime = System.currentTimeMillis();
		long concTime = endConcTime - initConcTime;
		threads.clear();
		return concTime;
	}


	private static long secuentialExecute() {
		long initSecTime = System.currentTimeMillis();
		for (int i = 0; i < n; i++) {
			for (int j = 0; j < n; j++) {
				secuencial[i][j] = inicial[i][j] * ESCALAR;
			}
		}
		long endSecTime = System.currentTimeMillis();
		return endSecTime - initSecTime;
	}


	private static void inicializarMatriz(int n) {
		inicial = new int[n][n];
		concurrente = new int[n][n];
		secuencial = new int[n][n];
		for (int i = 0; i < n; i++) {
			for (int j = 0; j < n; j++) {
				inicial[i][j] = new Random().nextInt(9);
			}
		}
	}


}


Writing hilos.java


In [ ]:
! java hilos.java 10

Las matrices secuencial y concurrente son iguales
Las matrices secuencial y concurrente son iguales
Las matrices secuencial y concurrente son iguales
Tiempo Secuencial : 0 milisegundos
Tiempo 1 Thread   : 9 milisegundos
Tiempo 2 Threads  : 12 milisegundos
Tiempo 4 Threads  : 17 milisegundos


In [ ]:
! java hilos.java 20

Las matrices secuencial y concurrente son iguales
Las matrices secuencial y concurrente son iguales
Las matrices secuencial y concurrente son iguales
Tiempo Secuencial : 1 milisegundos
Tiempo 1 Thread   : 14 milisegundos
Tiempo 2 Threads  : 22 milisegundos
Tiempo 4 Threads  : 33 milisegundos


In [ ]:
! java hilos.java 40

Las matrices secuencial y concurrente son iguales
Las matrices secuencial y concurrente son iguales
Las matrices secuencial y concurrente son iguales
Tiempo Secuencial : 1 milisegundos
Tiempo 1 Thread   : 5 milisegundos
Tiempo 2 Threads  : 21 milisegundos
Tiempo 4 Threads  : 21 milisegundos


In [ ]:
! java hilos.java 80

Las matrices secuencial y concurrente son iguales
Las matrices secuencial y concurrente son iguales
Las matrices secuencial y concurrente son iguales
Tiempo Secuencial : 0 milisegundos
Tiempo 1 Thread   : 2 milisegundos
Tiempo 2 Threads  : 1 milisegundos
Tiempo 4 Threads  : 1 milisegundos


In [ ]:
! java hilos.java 250

Las matrices secuencial y concurrente son iguales
Las matrices secuencial y concurrente son iguales
Las matrices secuencial y concurrente son iguales
Tiempo Secuencial : 4 milisegundos
Tiempo 1 Thread   : 8 milisegundos
Tiempo 2 Threads  : 12 milisegundos
Tiempo 4 Threads  : 15 milisegundos


In [ ]:
! java hilos.java 1000

Las matrices secuencial y concurrente son iguales
Las matrices secuencial y concurrente son iguales
Las matrices secuencial y concurrente son iguales
Tiempo Secuencial : 31 milisegundos
Tiempo 1 Thread   : 31 milisegundos
Tiempo 2 Threads  : 29 milisegundos
Tiempo 4 Threads  : 15 milisegundos


In [ ]:
! java hilos.java 10000

Las matrices secuencial y concurrente son iguales
Las matrices secuencial y concurrente son iguales
Las matrices secuencial y concurrente son iguales
Tiempo Secuencial : 129 milisegundos
Tiempo 1 Thread   : 144 milisegundos
Tiempo 2 Threads  : 123 milisegundos
Tiempo 4 Threads  : 99 milisegundos


In [ ]:
! java hilos.java 15000

Las matrices secuencial y concurrente son iguales
Las matrices secuencial y concurrente son iguales
Las matrices secuencial y concurrente son iguales
Tiempo Secuencial : 228 milisegundos
Tiempo 1 Thread   : 291 milisegundos
Tiempo 2 Threads  : 279 milisegundos
Tiempo 4 Threads  : 229 milisegundos


# C++

In [ ]:
%%writefile hilos.cpp
#include <iostream>
#include <vector>
#include <thread>
#include <random>
#include <chrono>
#include <cstdlib>

using namespace std;

int N_COLUMNAS_Y_FILAS = 0;
int N_HILOS = 0;
int ESCALAR = 2;

vector<vector<int>> GenerarMatriz()
{
    vector<vector<int>> matriz(N_COLUMNAS_Y_FILAS, vector<int>(N_COLUMNAS_Y_FILAS));

    for (int i = 0; i < N_COLUMNAS_Y_FILAS; ++i)
    {
        for (int j = 0; j < N_COLUMNAS_Y_FILAS; ++j)
        {
            matriz[i][j] = rand() % 10;
        }
    }

    return matriz;
}

vector<vector<int>> MultiplicarMatrizRS(vector<vector<int>> matriz)
{

    vector<vector<int>> matrizRS(N_COLUMNAS_Y_FILAS, vector<int>(N_COLUMNAS_Y_FILAS));

    for (int i = 0; i < N_COLUMNAS_Y_FILAS; ++i)
    {
        for (int j = 0; j < N_COLUMNAS_Y_FILAS; ++j)
        {
                matrizRS[i][j] += matriz[i][j] * ESCALAR;
        }
    }

    return matrizRS;
}

void MultiplicarFila(vector<vector<int>> &matriz, vector<vector<int>> &matrizRC, int inicio, int final)
{
    for (int i = inicio; i < final; i++)
        for (int j = 0; j < N_COLUMNAS_Y_FILAS; j++)
        {
            matrizRC[i][j] += matriz[i][j] * ESCALAR;
        }
}

vector<vector<int>> GenerarMatrizRC(vector<vector<int>> matriz)
{
    vector<thread> threads;

    div_t resultado = div(N_COLUMNAS_Y_FILAS, N_HILOS);

    vector<vector<int>> matrizRC(N_COLUMNAS_Y_FILAS, vector<int>(N_COLUMNAS_Y_FILAS));

    int cociente = resultado.quot;
    int resto = resultado.rem;

    int inicio = 0;
    int final = 0;

    for (int i = 0; i < N_HILOS; ++i)
    {
        if (i == N_HILOS - 1)
        {
            final += cociente + resto;
        }
        else
        {
            final += cociente;
        }

        threads.emplace_back(MultiplicarFila, ref(matriz),ref(matrizRC), inicio, final);

        inicio = final;


    }

    for (auto &thread : threads)
    {
        thread.join();
    }

    return matrizRC;
}

bool CompararMatrices(vector<vector<int>> matrizRS, vector<vector<int>> matrizRC){

    for (int i = 0; i < N_COLUMNAS_Y_FILAS; ++i)
    {
        for (int j = 0; j < N_COLUMNAS_Y_FILAS; ++j)
        {
            if(matrizRS[i][j] != matrizRC[i][j])
            {
                cout<< matrizRC[i][j] << " != " << matrizRS[i][j] << endl;
                return false;
            }
        }
    }
    return true;
}

int main(int argc, char *argv[])
{
    if (argc != 3)
    {
        std::cerr << "Uso: " << argv[0] << " <cantidad_filas> <cantidad_hilos>" << std::endl;
        return 0;
    }

    N_COLUMNAS_Y_FILAS = atoi(argv[1]);
    N_HILOS = atoi(argv[2]);

    vector<vector<int>> matriz = GenerarMatriz();

    auto start_sequential = chrono::steady_clock::now();
    vector<vector<int>> matrizRS = MultiplicarMatrizRS(matriz);
    auto end_sequential = chrono::steady_clock::now();
    chrono::duration<double> sequential_time = end_sequential - start_sequential;
    auto start_concurrent = chrono::steady_clock::now();
    vector<vector<int>> matrizRC = GenerarMatrizRC(matriz);
    auto end_concurrent = chrono::steady_clock::now();
    chrono::duration<double> concurrent_time = end_concurrent - start_concurrent;

    if(CompararMatrices(matrizRS, matrizRC))
    {
        cout << "Tiempo secuencial: " << sequential_time.count() << " segundos" << endl;
        cout << "Tiempo concurrente: " << concurrent_time.count() << " segundos" << endl;
    }
    else
    {
        cout << "Las matrices son diferentes" << endl;
    }

    return 0;
}

Overwriting hilos.cpp


In [ ]:
!g++ -o hilos.bin hilos.cpp

In [ ]:
!./hilos.bin 10 1

Tiempo secuencial: 1.0217e-05 segundos
Tiempo concurrente: 0.000161757 segundos


In [ ]:
!./hilos.bin 10 2

Tiempo secuencial: 1.0417e-05 segundos
Tiempo concurrente: 0.000807843 segundos


In [ ]:
!./hilos.bin 10 4

Tiempo secuencial: 1.0813e-05 segundos
Tiempo concurrente: 0.000743163 segundos


In [ ]:
!./hilos.bin 20 1

Tiempo secuencial: 2.2707e-05 segundos
Tiempo concurrente: 0.000185149 segundos


In [ ]:
!./hilos.bin 20 2

Tiempo secuencial: 2.5739e-05 segundos
Tiempo concurrente: 0.0011763 segundos


In [ ]:
!./hilos.bin 20 4

Tiempo secuencial: 2.6863e-05 segundos
Tiempo concurrente: 0.00376818 segundos


In [ ]:
!./hilos.bin 40 1

Tiempo secuencial: 5.8924e-05 segundos
Tiempo concurrente: 0.000233547 segundos


In [ ]:
!./hilos.bin 40 2

Tiempo secuencial: 6.9024e-05 segundos
Tiempo concurrente: 0.000296206 segundos


In [ ]:
!./hilos.bin 40 4

Tiempo secuencial: 6.0268e-05 segundos
Tiempo concurrente: 0.000518899 segundos


In [ ]:
!./hilos.bin 80 1

Tiempo secuencial: 0.000204216 segundos
Tiempo concurrente: 0.000939087 segundos


In [ ]:
!./hilos.bin 80 2

Tiempo secuencial: 0.000234915 segundos
Tiempo concurrente: 0.000433637 segundos


In [ ]:
!./hilos.bin 80 4

Tiempo secuencial: 0.000223132 segundos
Tiempo concurrente: 0.000528782 segundos






# Python

In [ ]:
%%writefile hilos.py

import threading
from random import randrange
import numpy
import sys
import argparse
import math
import time


n = 0
inicial = [[]]
concurrente = [[]]
secuencial = [[]]
ESCALAR = 5

def multiplyRows(id,filaDesde,filaHasta):
    global concurrente,inicial,ESCALAR,n
    for i in range(filaDesde,filaHasta + 1):
          for j in range(n):
            concurrente[i][j] = (inicial[i][j] * ESCALAR);


def concurrentExecute(threads) :
    global inicial,concurrente,secuencial,ESCALAR,n
    my_threads = []
    filasPorHilos = n / threads
    for i in range(1,threads + 1):
      filaDesde = math.ceil((i - 1) * filasPorHilos)
      filaHasta = math.ceil((i * filasPorHilos) -1)
      my_threads.append(threading.Thread(target=multiplyRows,args=(i,filaDesde,filaHasta)));

    for i in range(threads):
      my_threads[i].start()

    for i in range(threads):
      my_threads[i].join()

def checkValidityOfMatrix():
   global n
   for i in range(n):
      for j in range(n):
         if secuencial[i][j] != concurrente[i][j] :
           print("ERROR",i,j)
           break

def main():
    global inicial,concurrente,secuencial,ESCALAR,n
    parser = argparse.ArgumentParser("argumentos_matriz")
    parser.add_argument("n", help="", type=int)
    args = parser.parse_args()
    n = args.n
    inicial = numpy.zeros((args.n, args.n))
    concurrente = numpy.zeros((args.n, args.n))
    secuencial = numpy.zeros((args.n, args.n))

    for i in range(args.n):
      for j in range(args.n):
        inicial[i][j] = randrange(10)

    startSecuencial = time.time()
    for i in range(args.n):
      for j in range(args.n):
         secuencial[i][j] = (inicial[i][j] * ESCALAR);
    endSecuencial = time.time()
    print("Tiempo secuencial : ", (endSecuencial - startSecuencial) * 1000 , "    milisegundos")


    start1Thread = time.time()
    concurrentExecute(1)
    end1Thread = time.time()
    print("Tiempo 1 thread   : ",  (end1Thread - start1Thread) * 1000 , "    milisegundos")
    checkValidityOfMatrix()

    start2Threads = time.time()
    concurrentExecute(2)
    end2Threads = time.time()
    print("Tiempo 2 threads  : ", (end2Threads - start2Threads) * 1000 , "    milisegundos")
    checkValidityOfMatrix()

    start4Threads = time.time()
    concurrentExecute(4)
    end4Threads = time.time()
    print("Tiempo 4 threads  : ", (end4Threads - start4Threads) * 1000 , "    milisegundos")
    checkValidityOfMatrix()


if __name__=="__main__":
    main()

Writing hilos.py


In [ ]:
!python hilos.py 10

Tiempo secuencial :  0.06508827209472656     milisegundos
Tiempo 1 thread   :  0.42748451232910156     milisegundos
Tiempo 2 threads  :  0.39649009704589844     milisegundos
Tiempo 4 threads  :  0.5218982696533203     milisegundos


In [ ]:
!python hilos.py 20

Tiempo secuencial :  0.46324729919433594     milisegundos
Tiempo 1 thread   :  0.7917881011962891     milisegundos
Tiempo 2 threads  :  0.8273124694824219     milisegundos
Tiempo 4 threads  :  0.9648799896240234     milisegundos


In [ ]:
!python hilos.py 40

Tiempo secuencial :  0.9636878967285156     milisegundos
Tiempo 1 thread   :  1.3954639434814453     milisegundos
Tiempo 2 threads  :  1.3537406921386719     milisegundos
Tiempo 4 threads  :  1.6393661499023438     milisegundos


In [ ]:
!python hilos.py 80

Tiempo secuencial :  5.739688873291016     milisegundos
Tiempo 1 thread   :  4.463672637939453     milisegundos
Tiempo 2 threads  :  4.548311233520508     milisegundos
Tiempo 4 threads  :  4.715204238891602     milisegundos


In [ ]:
!python hilos.py 250

Tiempo secuencial :  49.53145980834961     milisegundos
Tiempo 1 thread   :  39.78109359741211     milisegundos
Tiempo 2 threads  :  38.82169723510742     milisegundos
Tiempo 4 threads  :  40.88473320007324     milisegundos


In [ ]:
!python hilos.py 1000

Tiempo secuencial :  655.1439762115479     milisegundos
Tiempo 1 thread   :  665.764570236206     milisegundos
Tiempo 2 threads  :  1250.0131130218506     milisegundos
Tiempo 4 threads  :  667.5403118133545     milisegundos


In [ ]:
!python hilos.py 5000

Tiempo secuencial :  18552.059173583984     milisegundos
Tiempo 1 thread   :  19540.358543395996     milisegundos
Tiempo 2 threads  :  18374.186515808105     milisegundos
Tiempo 4 threads  :  18382.7543258667     milisegundos
